In [11]:
import os
from typing import TypedDict
import time
from langgraph.graph import StateGraph, END

╔════════════════════════════════════════╗
║     LangGraph Import Structure         ║
╠════════════════════════════════════════╣
║                                        ║
║  langgraph.graph                       ║
║      ├── StateGraph (Class)            ║
║      │    └─ Creates workflow graphs   ║
║      └── END (Constant)                ║
║           └─ Marks graph termination   ║
║                                        ║
║  typing.TypedDict                      ║
║      └── For defining State structure  ║
║           └─ Type-safe state schema    ║     
║                                        ║
╚════════════════════════════════════════╝

In [12]:
class State(TypedDict):
    name: str
    greeting: str

def greet_node(state: State):
    """A node that creates a greeting from the name"""
    print(" 🔄 Processing in greet_node...")
    time.sleep(2)
    greeting = f"Hello, {state['name']}"
    return {"greeting": greeting}

def enhance_node(state: State):
    """A node that enhances the greeting"""
    print(" 🔄 Processing in enhance node...")
    time.sleep(2)
    enhanced = state["greeting"] + "How are you?"
    return {"greeting": enhanced}
    

 ┌─────────────────────────────────────────┐
 │  Understanding Nodes in LangGraph       │
 └─────────────────────────────────────────┘

     ┌─────────────────┐
     │  Initial State  │
     │ name: "Alice"   │
     │ greeting: ""    │
     └────────┬────────┘
              │
              ▼
     ┌─────────────────┐
     │   greet_node    │ ← Node is a function
     │  Takes state    │
     │  Returns:       │
     │  {greeting:...} │ ← Partial update
     └────────┬────────┘
              │ (LangGraph merges)
              ▼
     ┌─────────────────┐
     │  Updated State  │
     │ name: "Alice"   │ ← Unchanged
     │ greeting:"Hello"│ ← Updated
     └────────┬────────┘
              │
              ▼
     ┌─────────────────┐
     │  enhance_node   │ ← Another function
     │  Takes state    │
     │  Returns:       │
     │  {greeting:...} │ ← Another update
     └────────┬────────┘
              │
              ▼
     ┌─────────────────┐
     │   Final State   │
     │ name: "Alice"   │
     │ greeting:"Hello │
     │  ...How are you?"│
     └─────────────────┘

 KEY INSIGHT: Nodes are just functions!
 - Take state as input
 - Return partial updates
 - LangGraph handles merging

In [13]:
print("Testing node manually:\n")

initial_state = {"name": "Alice", "greeting": ""}
print(f"Initial state: {initial_state}")

print("\nCalling greet node...")
update1 = greet_node(initial_state)
print(f"Node returend: {update1}")

state_after_greet = {"name": "Alice", "greeting": update1["greeting"]}
print(f"State after greeting: {state_after_greet}")

print("\nCalling enhance node ...")
update2 = enhance_node(state_after_greet)
print(f"Node returned: {update2}")

final_state = {"name": "Alice", "greeting": update2["greeting"]}
print(f"Final state: {final_state}")

Testing node manually:

Initial state: {'name': 'Alice', 'greeting': ''}

Calling greet node...
 🔄 Processing in greet_node...
Node returend: {'greeting': 'Hello, Alice'}
State after greeting: {'name': 'Alice', 'greeting': 'Hello, Alice'}

Calling enhance node ...
 🔄 Processing in enhance node...
Node returned: {'greeting': 'Hello, AliceHow are you?'}
Final state: {'name': 'Alice', 'greeting': 'Hello, AliceHow are you?'}


#### Connecting nodes with edges

In [14]:
print("Building first graph\n")

workflow = StateGraph(State)

workflow.add_node("greet", greet_node)
workflow.add_node("enhance", enhance_node)

workflow.set_entry_point("greet")
workflow.add_edge("greet","enhance")
workflow.add_edge("enhance", END)

# Compile the graph
print("Compiling graph...")
app = workflow.compile()
print("✅ Graph compiled succesfully!\n")

print("Running the graph...")
result = app.invoke({"name":"Bob", "greeting":""})

print(f"\nFinal result: {result}\n")

Building first graph

Compiling graph...
✅ Graph compiled succesfully!

Running the graph...
 🔄 Processing in greet_node...
 🔄 Processing in enhance node...

Final result: {'name': 'Bob', 'greeting': 'Hello, BobHow are you?'}

